# 네이버 영화평 감성 분석

In [ ]:
# KoNLPy 설치 
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 45.1MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

--2021-04-18 01:41:26--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt.1’

ratings_train.txt.1 100%[===================>]  13.95M  38.4MB/s    in 0.4s    

2021-04-18 01:41:27 (38.4 MB/s) - ‘ratings_train.txt.1’ saved [14628807/14628807]



In [ ]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2021-04-18 01:42:03--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  24.8MB/s    in 0.2s    

2021-04-18 01:42:03 (24.8 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]



In [ ]:
!ls -l

total 19072
-rw-r--r-- 1 root root  4893335 Apr 18 01:42 ratings_test.txt
-rw-r--r-- 1 root root 14628807 Apr 18 01:41 ratings_train.txt
drwxr-xr-x 1 root root     4096 Apr  7 13:36 sample_data


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [ ]:
test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [ ]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

### 데이터 전처리

In [ ]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [ ]:
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [ ]:
# Null값 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [ ]:
# Null값 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [ ]:
# 0과 1의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋에도 적용

In [ ]:
test_df.document.nunique()

49157

In [ ]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [ ]:
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [ ]:
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

In [ ]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 텍스트 전처리

In [ ]:
# 한글 이외의 문자는 제거
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
# 영문이나 숫자로만 되어 있는 문장은 데이터가 없어짐
train_df['document'].replace('', np.nan, inplace=True)
train_df.isnull().sum()

id            0
document    391
label         0
dtype: int64

In [ ]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- 테스트 데이터 셋

In [ ]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'].replace('', np.nan, inplace=True)
test_df.isnull().sum()

id            0
document    162
label         0
dtype: int64

In [ ]:
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

### 한글 형태소 분석 - 토큰화

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [ ]:
okt.morphs('교도소 이야기구먼 솔직히 재미는 없다평점 조정', stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [ ]:
from tqdm.notebook import tqdm
X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    X_train.append(temp)

In [ ]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    X_test.append(temp)

In [ ]:
y_train = train_df.label.values
y_test = test_df.label.values

### Keras 인코딩

In [ ]:
# 정수 인코딩
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 35000
tokenizer = Tokenizer(num_words=max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#  X_train과 X_test의 모든 샘플의 길이를 동일하게 30으로 셋팅
max_len=30
# 전체 데이터의 길이는 30으로 맞춘다.
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

### LSTM 모델 정의/설정/학습/평가

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential([
    Embedding(max_words, 100),
    LSTM(128),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3500000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,617,377
Trainable params: 3,617,377
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='naver-lstm-best-model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=60,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/5
1944/1944 [==============================] - 215s 109ms/step - loss: 0.4425 - accuracy: 0.7871 - val_loss: 0.3490 - val_accuracy: 0.8456

Epoch 00001: val_loss improved from inf to 0.34898, saving model to naver-lstm-best-model.h5
Epoch 2/5
1944/1944 [==============================] - 207s 106ms/step - loss: 0.2831 - accuracy: 0.8799 - val_loss: 0.3402 - val_accuracy: 0.8516

Epoch 00002: val_loss improved from 0.34898 to 0.34019, saving model to naver-lstm-best-model.h5
Epoch 3/5
1944/1944 [==============================] - 205s 105ms/step - loss: 0.2241 - accuracy: 0.9087 - val_loss: 0.3693 - val_accuracy: 0.8437

Epoch 00003: val_loss did not improve from 0.34019
Epoch 4/5
1944/1944 [==============================] - 204s 105ms/step - loss: 0.1750 - accuracy: 0.9310 - val_loss: 0.4031 - val_accuracy: 0.8455

Epoch 00004: val_loss did not improve from 0.34019
Epoch 5/5
1944/1944 [==============================] - 204s 105ms/step - loss: 0.1354 - accuracy: 0.9474 - val_loss:

In [ ]:
# Best Model 선택
from tensorflow.keras.models import load_model
best_model = load_model('naver-lstm-best-model.h5')

In [ ]:
_, acc = best_model.evaluate(X_test, y_test, verbose=2)
print(f'Accuracy: {acc:.4f}')

1532/1532 - 17s - loss: 0.3493 - accuracy: 0.8482
Accuracy: 0.8482
